In [1]:
using JLD2
using Statistics
using Revise
using Plots 
using Laplacians

In [2]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

nanErr! (generic function with 1 method)

In [4]:
@load "wted_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic1 = copy(dic)
@load "wted_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic2 = copy(dic)
@load "wted_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic3 = copy(dic)
@load "wted_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic4 = copy(dic)

Dict{Any,Any} with 44 entries:
  "hypre_solve" => [933.33, 160.63, Inf, 72.56, 256.45, 123.59, 195.46, 199.09]
  "icc2_tot"    => [848.565, 397.734, 45.3928, 1110.36, 1544.77, 882.98, 884.11…
  "lamg_its"    => [Inf, Inf, Inf, 2.0, 11.0, 9.0, Inf, Inf]
  "ac_tot"      => [220.359, 197.952, 134.363, 174.693, 471.167, 312.723, 436.1…
  "icc2_solve"  => [833.318, 378.335, 29.3293, 1092.6, 1524.96, 860.902, 863.36…
  "hypre_tot"   => [1159.99, 413.61, Inf, 186.37, 612.68, 326.37, 438.17, 432.5…
  "ac-s2m2_its" => [34.0, 17.0, 21.0, 26.0, 28.0, 20.0, 36.0, 22.0]
  "icc_solve"   => [744.837, 323.045, 21.7609, 748.333, 1255.83, 812.707, 634.6…
  "icc2_its"    => [743.0, 261.0, 23.0, 958.0, 987.0, 517.0, 506.0, 540.0]
  "cmg2_solve"  => [159.606, 286.223, 204.283, 136.845, 229.339, 313.861, 188.2…
  "icc_build"   => [19.8467, 30.5808, 19.2364, 24.2287, 36.4185, 35.7064, 32.24…
  "cmg_err"     => [9.32752e-9, 7.59333e-9, 9.447e-9, 6.74696e-9, 9.14326e-9, 9…
  "lamg_err"    => [Inf, Inf, Inf, In

In [5]:
nanErr!(dic1);
nanErr!(dic2);
nanErr!(dic3);
nanErr!(dic4);

In [6]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)

ffmicro (generic function with 1 method)

In [7]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [8]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(nfK(d["nv"][1]), "&", length(d["testName"]), "&", join(allCells, "&"), "\\\\"))
end

function printAllLines(dics, solvers)
    for d in dics
        printOneLine(d, solvers)
    end
end

printAllLines (generic function with 1 method)

In [9]:
dics = [dic1, dic2, dic3, dic4];

In [10]:
solvers=["ac", "ac-s2m2", "cmg2"]
printAllLines(dics, solvers)

10&103&0.365&0.405&0.455&0.572&0.729&0.939&8.09&11.1&22\\
100&105&0.605&0.745&1.31&0.908&1.26&2.25&1.84&2.51&Inf\\
1000&23&1.5&1.8&2.19&1.67&2.53&3.37&1.84&2.65&Inf\\
10000&8&3.16&4.21&4.39&3.83&4.19&4.79&3.21&3.55&6.22\\


In [11]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dics, solvers)

10&103&0.968&1.36&5.18\tnote{**}&4.58&11.2&32.9\tnote{*}&4.71&5.82&Inf\\
100&105&1.25&2.3&7.7\tnote{**}&12.9&33.3&151\tnote{*}&1.9&3.35&Inf\\
1000&23&3.24&5.8&15.8\tnote{*}&52.5&218&861\tnote{*}&4.46&10.7&Inf\\
10000&8&4.88&10.7\tnote{**}&Inf&N/A&N/A&N/A&9.03&14.5\tnote{*}&17\tnote{*}\\
